In [90]:
# 使用PyG 简单实现 GCN

In [91]:
import torch
from torch_geometric.datasets import Planetoid, NELL
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


In [92]:
data=Planetoid(root='../data/CiteSeer',name='CiteSeer')
print(len(data))

1


In [93]:
data=data[0]
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [94]:
print(data.is_directed())

False


In [95]:
data.y  #表示节点的标签编号

tensor([3, 1, 5,  ..., 3, 1, 5])

In [96]:
data.train_mask #若为True 则表示该节点为训练节点

tensor([ True,  True,  True,  ..., False, False, False])

In [97]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 32)
        self.conv2 = GCNConv(32, num_classes)
        self.norm = torch.nn.BatchNorm1d(32)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.norm(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x


In [98]:
def train(model,data,device):
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=1e-4)
    loss_fun=torch.nn.CrossEntropyLoss().to(device=device)
    model.train()

    for epoch in range(200):
        out=model(data)
        optimizer.zero_grad()
        loss=loss_fun(out[data.train_mask],data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        print('Epoch {:03d} loss{: .4f}'.format(epoch,loss.item()))

    

In [99]:
def test(model,data):
    model.eval()
    _,pred=model(data).max(dim=1)
    correct=int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc=correct/int(data.test_mask.sum())

    print('GCN Accuracy {:.4f}'.format(acc))

In [100]:
def load_data(name):
    if name=='NELL':
        print('../data/'+name+'/')
        dataset=NELL(root='../data/'+name+'/')
    else:
        dataset = Planetoid(root='../data/' + name + '/', name=name)
    
    _device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    data=dataset[0].to(_device)
    if name=='NELL':
        data.x=data.x.to_dense()
        num_node_features=dataset.x.shape[1]
    else:
        num_node_features=dataset.num_node_features
    
    return data,num_node_features,dataset.num_classes


In [101]:
def main():
    names = ['CiteSeer', 'Cora', 'PubMed']
    for name in names:
        print(name+'...')
        data,num_node_features,num_classes=load_data(name)
        print(data,num_node_features,num_classes)
        _device ='cuda'
        device = torch.device(_device)
        model=GCN(num_node_features,num_classes).to(device)
        train(model,data,device)
        test(model,data)

In [102]:
if __name__=='__main__':
    main()

CiteSeer...
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327]) 3703 6


Epoch 000 loss 2.1580
Epoch 001 loss 0.7630
Epoch 002 loss 0.4595
Epoch 003 loss 0.3265
Epoch 004 loss 0.2377
Epoch 005 loss 0.1595
Epoch 006 loss 0.1622
Epoch 007 loss 0.1186
Epoch 008 loss 0.0941
Epoch 009 loss 0.0794
Epoch 010 loss 0.0832
Epoch 011 loss 0.0538
Epoch 012 loss 0.0645
Epoch 013 loss 0.0596
Epoch 014 loss 0.0360
Epoch 015 loss 0.0656
Epoch 016 loss 0.0402
Epoch 017 loss 0.0387
Epoch 018 loss 0.0288
Epoch 019 loss 0.0619
Epoch 020 loss 0.0183
Epoch 021 loss 0.0304
Epoch 022 loss 0.0231
Epoch 023 loss 0.0244
Epoch 024 loss 0.0134
Epoch 025 loss 0.0258
Epoch 026 loss 0.0118
Epoch 027 loss 0.0481
Epoch 028 loss 0.0160
Epoch 029 loss 0.0261
Epoch 030 loss 0.0313
Epoch 031 loss 0.0156
Epoch 032 loss 0.0084
Epoch 033 loss 0.0077
Epoch 034 loss 0.0210
Epoch 035 loss 0.0192
Epoch 036 loss 0.0154
Epoch 037 loss 0.0298
Epoch 038 loss 0.0140
Epoch 039 loss 0.0367
Epoch 040 loss 0.0162
Epoch 041 loss 0.0103
Epoch 042 loss 0.0186
Epoch 043 loss 0.0080
Epoch 044 loss 0.0114
Epoch 045 

Using existing file ind.cora.x
Processing...
Done!


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708]) 1433 7
Epoch 000 loss 2.2459
Epoch 001 loss 1.0093
Epoch 002 loss 0.6108
Epoch 003 loss 0.4078
Epoch 004 loss 0.2920
Epoch 005 loss 0.2372
Epoch 006 loss 0.2209
Epoch 007 loss 0.1855
Epoch 008 loss 0.1353
Epoch 009 loss 0.1217
Epoch 010 loss 0.0924
Epoch 011 loss 0.1022
Epoch 012 loss 0.0646
Epoch 013 loss 0.0682
Epoch 014 loss 0.0473
Epoch 015 loss 0.0455
Epoch 016 loss 0.0420
Epoch 017 loss 0.0354
Epoch 018 loss 0.0328
Epoch 019 loss 0.0391
Epoch 020 loss 0.0301
Epoch 021 loss 0.0196
Epoch 022 loss 0.0195
Epoch 023 loss 0.0225
Epoch 024 loss 0.0228
Epoch 025 loss 0.0125
Epoch 026 loss 0.0222
Epoch 027 loss 0.0082
Epoch 028 loss 0.0086
Epoch 029 loss 0.0174
Epoch 030 loss 0.0166
Epoch 031 loss 0.0090
Epoch 032 loss 0.0102
Epoch 033 loss 0.0061
Epoch 034 loss 0.0066
Epoch 035 loss 0.0057
Epoch 036 loss 0.0060
Epoch 037 loss 0.0050
Epoch 038 loss 0.0058
Epoch 039 loss 0.0097
Epoch